In [1]:
import torch
import importlib
import sys
#importlib.reload(sys.modules['fairseq.models.roberta'])
from fairseq.models.roberta import RobertaModel

In [2]:
import random

In [3]:
pretrain = RobertaModel.from_pretrained(
    '/new_home/zhuocheng/transformer/models/roberta.large',
    checkpoint_file='model.pt',
).cuda()
pretrain.eval()

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): FusedLayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (v_proj): NogradLinear(in_features=1024, out_features=1024, bias=True)
              (out_proj): NogradLinear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): FusedLayerNorm(torch.Size([1024]), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1): NogradLin

In [4]:
train_data = []
with open('/new_home/zhuocheng/transformer/datasets/glue_data/CoLA/train.tsv', 'r', encoding='utf-8') as fin:
    for line in fin:
        train_data.append(line)


In [14]:
a = random.sample(train_data, 2)[0]
print(a)
b = a.strip().split('\t')[-1]
print(b)
c = pretrain.encode(b).cuda()
print(c)

r-67	0	*	Did that he played the piano surprise you?

Did that he played the piano surprise you?
tensor([    0, 20328,    14,    37,   702,     5, 13305,  2755,    47,   116,
            2], device='cuda:0')


In [11]:
train_data = []
with open('/new_home/zhuocheng/transformer/datasets/glue_data/CoLA/train.tsv', 'r', encoding='utf-8') as fin:
    for line in fin:
        train_data.append(line)

train_attn = []
cnt = 0
with torch.no_grad():
    for i in train_data:
        sent = i.strip().split('\t')[-1]
        tokens = pretrain.encode(sent).cuda()
        attn = pretrain.model.encoder.sentence_encoder.forward_scriptable(tokens.unsqueeze(0))['attn_weights']
        attns = torch.stack(attn).squeeze()
        train_attn.append(attns.cpu())

In [34]:
valid_data = []
with open('/new_home/zhuocheng/transformer/datasets/glue_data/CoLA/dev.tsv', 'r', encoding='utf-8') as fin:
    for line in fin:
        valid_data.append(line)

valid_attn = []
cnt = 0
with torch.no_grad():
    for i in valid_data:
        sent = i.strip().split('\t')[-1]
        tokens = pretrain.encode(sent).cuda()
        attn = pretrain.model.encoder.sentence_encoder.forward_scriptable(tokens.unsqueeze(0))['attn_weights']
        attns = torch.stack(attn).squeeze()
        valid_attn.append(attns.cpu())

torch.save(valid_attn, 'CoLA-bin/input0/valid.attn')

In [35]:
torch.save(valid_attn, 'CoLA-bin/input0/valid.attn')

In [59]:
t = pretrain.model.encoder.sentence_encoder.forward_scriptable(tokens.unsqueeze(0))['attn_weights']

In [61]:
t[0].shape

torch.Size([16, 1, 9, 9])

In [9]:
tokens = random.sample(train_data, 1)[0].strip().split('\t')
sent, target = tokens[-1], tokens[1]
print(sent, target)
tokens = pretrain.encode(sent)

Everyone claimed that the poison was neutralized. 1


In [11]:
pre_attn = pretrain.model.encoder.sentence_encoder.forward_scriptable(tokens.unsqueeze(0))['attn_weights']

In [14]:
print(len(pre_attn))
print(pre_attn[0].squeeze().shape)

24
torch.Size([16, 11, 11])


In [19]:
torch.stack(pre_attn).squeeze()

torch.Size([24, 16, 11, 11])

In [15]:
from scipy.stats import wasserstein_distance

In [16]:
def sim_basic(a, b):
    res = 0
    for i in range(len(a)):
        res += wasserstein_distance(a[i], b[i])
    return res

In [21]:
for layer in range(24):
    a = rft_attn[layer].squeeze().tolist()
    b = pre_attn[layer].squeeze().tolist()
    c = fft_attn[layer].squeeze().tolist()
    res_ab = res_ac = res_bc = 0
    for i in range(16):
        ab = sim_basic(a[i], b[i])
        ac = sim_basic(a[i], c[i])
        bc = sim_basic(b[i], c[i])
        res_ab += ab
        res_ac += ac
        res_bc += bc
    print(layer, res_ab, res_ac, res_bc)

0 0.13261233243331144 1.7378198558153632 1.752287221363657
1 0.971153716215498 2.2803670261976055 2.1397048345580743
2 0.5554521750420954 1.9973413279025445 1.9882324791034671
3 1.0139972043634184 3.139335905933267 2.67003177155075
4 0.9823197402890138 3.916214081658086 3.7933111355767752
5 1.208766217504394 5.461882534223789 5.362139308712325
6 1.3082826459248218 4.597824672018476 4.387153685950609
7 1.449606107705581 5.816301996011135 5.8190094251543965
8 1.279245120924543 4.294332645427961 4.218336600911167
9 1.518400905302722 7.5710816297505845 7.29659077263653
10 1.9318525922317527 5.341577278688 4.867779939222645
11 1.5309769671502733 6.466154411870743 6.218709221485614
12 1.5887899272335362 6.343949524410581 5.85831688233507
13 1.5055914101594086 7.032440083455869 6.230813680211622
14 2.0266159083046085 6.542214888008692 5.578992886559695
15 1.460632795942825 5.309213703531346 5.266570026429668
16 1.9567968369653839 6.34970954800327 5.794015902121721
17 2.1422264220333385 5.7919

In [15]:
train_attn = torch.load('CoLA-bin/input0/train.attn')
train_mean = []
for i in train_attn:
    train_mean.append(torch.softmax(i.new(i.shape).fill_(1), dim=-1))
torch.save(train_mean, 'CoLA-bin/input0/train.mean')
valid_attn = torch.load('CoLA-bin/input0/valid.attn')
valid_mean = []
for i in valid_attn:
    valid_mean.append(torch.softmax(i.new(i.shape).fill_(1), dim=-1))
torch.save(valid_mean, 'CoLA-bin/input0/valid.mean')

In [19]:
train_attn = torch.load('CoLA-bin/input0/train.attn')
train_rand = []
for i in train_attn:
    train_rand.append(torch.softmax(torch.rand(i.shape), dim=-1))
torch.save(train_rand, 'CoLA-bin/input0/train.rand')
valid_attn = torch.load('CoLA-bin/input0/valid.attn')
valid_rand = []
for i in valid_attn:
    valid_rand.append(torch.softmax(torch.rand(i.shape), dim=-1))
torch.save(valid_rand, 'CoLA-bin/input0/valid.rand')

In [21]:
train_rand[0]

tensor([[[[0.0366, 0.0551, 0.0476,  ..., 0.0373, 0.0478, 0.0431],
          [0.0425, 0.0719, 0.0814,  ..., 0.0615, 0.0859, 0.0528],
          [0.0475, 0.0719, 0.0353,  ..., 0.0922, 0.0638, 0.0368],
          ...,
          [0.0442, 0.0411, 0.0826,  ..., 0.0730, 0.0801, 0.0354],
          [0.0419, 0.0744, 0.0453,  ..., 0.0808, 0.0382, 0.0715],
          [0.0475, 0.0594, 0.0421,  ..., 0.0647, 0.0515, 0.0660]],

         [[0.0457, 0.0663, 0.0426,  ..., 0.0733, 0.0553, 0.0418],
          [0.0410, 0.0463, 0.0472,  ..., 0.0569, 0.0645, 0.0445],
          [0.0593, 0.0407, 0.0806,  ..., 0.0375, 0.0361, 0.0552],
          ...,
          [0.0377, 0.0818, 0.0353,  ..., 0.0719, 0.0576, 0.0472],
          [0.0377, 0.0454, 0.0615,  ..., 0.0401, 0.0472, 0.0515],
          [0.0505, 0.0485, 0.0401,  ..., 0.0379, 0.0536, 0.0781]],

         [[0.0334, 0.0337, 0.0678,  ..., 0.0513, 0.0438, 0.0788],
          [0.0414, 0.0726, 0.0561,  ..., 0.0367, 0.0433, 0.0407],
          [0.0690, 0.0382, 0.0566,  ..., 0